In [0]:
from numpy import array
import numpy as np
from keras.callbacks import CSVLogger

from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
import io

In [0]:
# load doc into memory
def load_doc(filename):
    with io.open(filename, 'r', encoding='utf8') as f:
        text = f.read()
    return text

# load
in_filename = r'updated_data_4_1.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [18]:
lines

['\ufeffকর্পোরেশনের দোষ তো আছে ।',
 'দোষ তো আছে । কিন্তু',
 'তো আছে । কিন্তু আমরা',
 'আছে । কিন্তু আমরা যে',
 '। কিন্তু আমরা যে সুযোগ',
 'কিন্তু আমরা যে সুযোগ বুঝে',
 'আমরা যে সুযোগ বুঝে হাজার',
 'যে সুযোগ বুঝে হাজার হাজার',
 'সুযোগ বুঝে হাজার হাজার কোটি',
 'বুঝে হাজার হাজার কোটি টাকার',
 'হাজার হাজার কোটি টাকার ট্যাক্স',
 'হাজার কোটি টাকার ট্যাক্স ফাকি',
 'কোটি টাকার ট্যাক্স ফাকি দিয়ে',
 'টাকার ট্যাক্স ফাকি দিয়ে আসছি',
 'ট্যাক্স ফাকি দিয়ে আসছি সেটাও',
 'ফাকি দিয়ে আসছি সেটাও আমরা',
 'দিয়ে আসছি সেটাও আমরা জানি',
 'আসছি সেটাও আমরা জানি ।',
 'সেটাও আমরা জানি । এই',
 'আমরা জানি । এই ভয়ে',
 'জানি । এই ভয়ে সবাই',
 '। এই ভয়ে সবাই স্যু',
 'এই ভয়ে সবাই স্যু করার',
 'ভয়ে সবাই স্যু করার সাহস',
 'সবাই স্যু করার সাহস পাবেনা',
 'স্যু করার সাহস পাবেনা ।',
 'করার সাহস পাবেনা । কারণ',
 'সাহস পাবেনা । কারণ উলটা',
 'পাবেনা । কারণ উলটা স্যু',
 '। কারণ উলটা স্যু সে',
 'কারণ উলটা স্যু সে নিজে',
 'উলটা স্যু সে নিজে খেয়ে',
 'স্যু সে নিজে খেয়ে যাবে',
 'সে নিজে খেয়ে যাবে ।',
 'নিজে খেয়ে যাবে । আপনি',
 'খেয়ে যাব

In [0]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [20]:
sequences

[[10369, 2568, 49, 20, 1],
 [2568, 49, 20, 1, 18],
 [49, 20, 1, 18, 92],
 [20, 1, 18, 92, 17],
 [1, 18, 92, 17, 280],
 [18, 92, 17, 280, 512],
 [92, 17, 280, 512, 400],
 [17, 280, 512, 400, 400],
 [280, 512, 400, 400, 3419],
 [512, 400, 400, 3419, 1379],
 [400, 400, 3419, 1379, 3420],
 [400, 3419, 1379, 3420, 3421],
 [3419, 1379, 3420, 3421, 45],
 [1379, 3420, 3421, 45, 1380],
 [3420, 3421, 45, 1380, 669],
 [3421, 45, 1380, 669, 92],
 [45, 1380, 669, 92, 456],
 [1380, 669, 92, 456, 1],
 [669, 92, 456, 1, 4],
 [92, 456, 1, 4, 1950],
 [456, 1, 4, 1950, 135],
 [1, 4, 1950, 135, 1381],
 [4, 1950, 135, 1381, 84],
 [1950, 135, 1381, 84, 670],
 [135, 1381, 84, 670, 3422],
 [1381, 84, 670, 3422, 1],
 [84, 670, 3422, 1, 153],
 [670, 3422, 1, 153, 3423],
 [3422, 1, 153, 3423, 1381],
 [1, 153, 3423, 1381, 11],
 [153, 3423, 1381, 11, 610],
 [3423, 1381, 11, 610, 382],
 [1381, 11, 610, 382, 106],
 [11, 610, 382, 106, 1],
 [610, 382, 106, 1, 35],
 [382, 106, 1, 35, 1381],
 [106, 1, 35, 1381, 3],
 [1

In [0]:
length = 5

In [22]:
print(len(sequences))
print(sequences[0][0])

60234
10369


In [23]:
sequences = array(sequences)
sequences

array([list([10369, 2568, 49, 20, 1]), list([2568, 49, 20, 1, 18]),
       list([49, 20, 1, 18, 92]), ..., list([2375, 1329, 484, 1, 10368]),
       list([1329, 484, 1, 10368, 4]), list([484, 1, 10368, 4, 10370])],
      dtype=object)

In [24]:
c=1
h=0
while(c!=0):
    limit = sequences.size
    d =[]
    c=0
    for i in range(0 , limit):
        l=len(sequences[i])
        if(l!=length):
            print(sequences[i])
            sequences = np.delete(sequences, i, 0)
            
            c=1
            print("error at ", i)
            print("Deleting :_________________________ ") 
            h = h+1
            break        
print("Total deletion: ", h)     

[70, 7, 979, 20]
error at  6666
Deleting :_________________________ 
[7, 979, 20, 197]
error at  6666
Deleting :_________________________ 
[979, 20, 197, 6]
error at  6666
Deleting :_________________________ 
[20, 197, 6, 237]
error at  6666
Deleting :_________________________ 
[197, 6, 237, 62]
error at  6666
Deleting :_________________________ 
[593, 10, 85, 148]
error at  7717
Deleting :_________________________ 
[10, 85, 148, 6]
error at  7717
Deleting :_________________________ 
[85, 148, 6, 4032]
error at  7717
Deleting :_________________________ 
[148, 6, 4032, 4033]
error at  7717
Deleting :_________________________ 
[6, 4032, 4033, 701]
error at  7717
Deleting :_________________________ 
[1267, 69, 2, 24]
error at  7820
Deleting :_________________________ 
[69, 2, 24, 24]
error at  7820
Deleting :_________________________ 
[2, 24, 24, 85]
error at  7820
Deleting :_________________________ 
[24, 24, 85, 174]
error at  7820
Deleting :_________________________ 
[24, 85, 174, 1]
e

In [25]:
print(sequences[564])
print(sequences.size)

[2591, 238, 1394, 1, 37]
58205


In [26]:
sequences = np.array(sequences.tolist())
sequences 

array([[10369,  2568,    49,    20,     1],
       [ 2568,    49,    20,     1,    18],
       [   49,    20,     1,    18,    92],
       ...,
       [ 2375,  1329,   484,     1, 10368],
       [ 1329,   484,     1, 10368,     4],
       [  484,     1, 10368,     4, 10370]])

In [27]:
sequences.shape

(58205, 5)

In [0]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [29]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=seq_length))
model.add(GRU(100, return_sequences=True))
model.add(GRU(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
csv_logger = CSVLogger('training_4_1.csv')
model.fit(X, y, batch_size=100, epochs=500,validation_split= 0.2 , callbacks=[csv_logger])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 50)             518550    
_________________________________________________________________
gru_3 (GRU)                  (None, 4, 100)            45300     
_________________________________________________________________
gru_4 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 10371)             1047471   
Total params: 1,681,721
Trainable params: 1,681,721
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1000
58205/58205 [==============================] - 5s 89us/step - loss: 8.0560 - acc: 0.1170
Epoch 2/1000
58205/58205 

In [0]:
# save the model to file
model.save('model_4_1.h5')
#model.save(r'C:\Users\OFR\Desktop\Thesis work\version three\model_4_1.h5')
# save the tokenizer
dump(tokenizer, open(r'tokenizer_4_1.pkl', 'wb'))